In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np
from keras import models, layers, Model
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import cv2

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 224
CHANNELS = 3
EPOCHS = 50
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
CLASS_NAMES = ['label 0', 'label 1', 'label 2']
N_CLASSES = len(CLASS_NAMES)

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'Dataset path',
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
plt.figure(figsize=(10,10))
for image_batch, label_batch in dataset.take(1):
    for i in range(4):
        ax = plt.subplot(3, 4, i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(class_names[label_batch[i]])
        plt.axis('off')

In [ ]:
def partition_data(ds, train_split = 0.8, val_split = 0.1, test_split = 0.1, shuffle = True, shuffle_size = 10000):
    ds_size = len(ds)
    if shuffle:
        ds  = ds.shuffle(shuffle_size, seed = 12)
    train_size = int(train_split * ds_size)
    validation_size = int(val_split * ds_size)
    train_data = ds.take(train_size)
    validation_data = ds.skip(train_size).take(validation_size)
    test_data = ds.skip(train_size).skip(validation_size)
    return train_data, validation_data, test_data

train_data, validation_data, test_data = partition_data(dataset)
print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(validation_data)}")
print(f"Test data size: {len(test_data)}")

In [ ]:
def prepare(ds):
    return ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

train_data = prepare(train_data)
validation_data = prepare(validation_data)
test_data = prepare(test_data)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255),
])

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(0.2, 0.2),
])

In [ ]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(16, (3,3), activation='relu', input_shape = input_shape),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')
])
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
model.build(input_shape=input_shape)

In [ ]:


def build_mobilenet_model():
    base = tf.keras.applications.MobileNetV2(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')
    base.trainable = False
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(N_CLASSES, activation='softmax')
    ])
    return model


In [ ]:
def build_resnet50_model():
    base = tf.keras.applications.ResNet50(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')
    base.trainable = False
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(N_CLASSES, activation='softmax')
    ])
    return model

In [ ]:
# Model builders
def build_vgg19_model():
    base = tf.keras.applications.VGG19(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')
    base.trainable = False
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(N_CLASSES, activation='softmax')
    ])
    return model

In [ ]:
def build_efficientnet_model():
    base = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')
    base.trainable = False
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        # layers.Dropout(0.3),
        layers.Dense(N_CLASSES, activation='softmax')
    ])
    return model

In [ ]:
# call the model
model = build_efficientnet_model()

#model = build_mobilenet_model()
# model = build_resnet50_model()

In [ ]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=validation_data
)

In [ ]:
scores = model.evaluate(test_data)
scores

In [ ]:
y_true = []
y_pred = []
for images, labels in test_data:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))
print("F1 Score (macro):", f1_score(y_true, y_pred, average='macro'))
print("F1 Score (weighted):", f1_score(y_true, y_pred, average='weighted'))
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:

plt.figure(figsize = (10,8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), accuracy, label = "Trainnign Accuracy")
plt.plot(range(EPOCHS), val_accuracy, label = "Validation Accuracy")
plt.legend(loc = 'lower right')
plt.title("Trainnign & Validation Accuracy")
plt.show()
plt.figure(figsize = (10,8))
plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label = "Trainnign Loss")
plt.plot(range(EPOCHS), val_loss, label = "Validation Loss")
plt.legend(loc = 'upper right')
plt.title("Trainnign & Validation Loss")
plt.show()

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras import models, layers
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# =====================
# Config
# =====================
BATCH_SIZE = 32
IMAGE_SIZE = 224
CHANNELS = 3
EPOCHS = 50
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
CLASS_NAMES = ['label 0', 'label 1', 'label 2']  # optional, will read from folder
AUG_TIMES = 20                                   # <-- Each training sample appears 20× per epoch
DATA_DIR = r"Dataset path"                       # <-- change to your dataset root

# =====================
# Load dataset (batched)
# =====================
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
class_names = dataset.class_names
print("Found classes:", class_names)

# Quick visual check
plt.figure(figsize=(8, 6))
for image_batch, label_batch in dataset.take(1):
    for i in range(min(4, image_batch.shape[0])):
        ax = plt.subplot(2, 2, i + 1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(class_names[int(label_batch[i])])
        plt.axis('off')
plt.tight_layout()
plt.show()

# =====================
# Split: train/val/test using cardinality
# =====================
def partition_data(ds, train_split=0.8, val_split=0.1, test_split=0.1):
    assert abs(train_split + val_split + test_split - 1.0) < 1e-6, "Splits must sum to 1.0"
    # number of BATCHES in ds
    n_batches = tf.data.experimental.cardinality(ds).numpy()
    if n_batches == tf.data.experimental.INFINITE_CARDINALITY:
        raise ValueError("Dataset has infinite cardinality. Remove repeats before partitioning.")
    if n_batches < 3:
        raise ValueError("Not enough batches to split. Reduce batch size or gather more data.")

    n_train = int(n_batches * train_split)
    n_val = int(n_batches * val_split)
    n_test = n_batches - n_train - n_val

    train_ds = ds.take(n_train)
    val_ds = ds.skip(n_train).take(n_val)
    test_ds = ds.skip(n_train + n_val).take(n_test)
    return train_ds, val_ds, test_ds

train_data, validation_data, test_data = partition_data(dataset)
print(f"Train batches: {tf.data.experimental.cardinality(train_data).numpy()}")
print(f"Val batches:   {tf.data.experimental.cardinality(validation_data).numpy()}")
print(f"Test batches:  {tf.data.experimental.cardinality(test_data).numpy()}")

# =====================
# Prepare pipelines
# =====================
def prepare(ds, cache=True):
    if cache:
        ds = ds.cache()
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

validation_data = prepare(validation_data)
test_data = prepare(test_data)

# === Make each original training image appear 20× per epoch (ONLINE augmentation) ===
def repeat_for_aug(ds, times=AUG_TIMES, batch_size=BATCH_SIZE):
    # Unbatch to single examples, replicate each example `times`, then re-batch
    return (ds
            .unbatch()
            .flat_map(lambda x, y: tf.data.Dataset.from_tensors((x, y)).repeat(times))
            .shuffle(1000, seed=42, reshuffle_each_iteration=True)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))

train_data = repeat_for_aug(train_data, AUG_TIMES, BATCH_SIZE)

# =====================
# Layers: resize/rescale + ONLINE augmentation
# =====================
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255),
], name="resize_rescale")

# You can adjust ranges if you want. These are the ones you used:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(0.2, 0.2),
], name="online_aug")

# =====================
# Model
# =====================
model = models.Sequential([
    layers.Input(shape=INPUT_SHAPE),
    resize_and_rescale,
    data_augmentation,                     # <-- random transforms applied during training
    layers.Conv2D(16, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()

# =====================
# Train
# =====================
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=EPOCHS
)

# =====================
# Evaluate
# =====================
print("\nEvaluating on test set...")
test_loss, test_acc = model.evaluate(test_data)
print(f"Test accuracy: {test_acc:.4f}")

# =====================
# Classification report & confusion matrix
# =====================
y_true = []
y_pred = []

for images, labels in test_data:
    preds = model.predict(images, verbose=0)
    y_true.extend(labels.numpy().tolist())
    y_pred.extend(np.argmax(preds, axis=1).tolist())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(xticks_rotation=45)
plt.tight_layout()
plt.show()
